# MNIST Syft Data Scientist

In [ ]:
%%capture
# this only runs in colab and clones the code sets it up and fixes a few issues
import sys
if "google.colab" in sys.modules:
    branch = "demo"    # change to the branch you want
    ! git clone --single-branch --branch $branch https://github.com/OpenMined/PySyft.git
    ! cd PySyft && ./scripts/colab.sh.     # fixes some colab python issues
    sys.path.append("/content/PySyft/src") # prevents needing restart

In [ ]:
import syft as sy
sy.LOG_FILE = "syft_ds.log"
_ = sy.logger.add(sy.LOG_FILE, enqueue=True, colorize=False, diagnose=True, backtrace=True, level="TRACE")

# PART 1: Connect to a Remote Duet Server

As the data scientist, you want to perform data science on data that is sitting in the Data Owner's Duet server (in their Notebook).

In order to do this, we must run the code that the Data Owner sends us, which importantly includes their Duet Session ID. This will create a direct connection from my notebook to the remote Duet server. Once the connection is established all traffic is sent directly between the two nodes.

Let's run the code below and follow the instructions it gives.

In [ ]:
# duet = sy.join_duet(loopback=True)
duet = sy.duet('e47f546963b1d213a20c7520d5174c3e')



# PART 2: Launch a Duet Server and Connect

In [ ]:
duet.store.pandas

In [ ]:
x = duet.store["age_data"]

In [ ]:
x = duet.syft.lib.python.Int(1)
print(x)

In [ ]:
%%time
sy.logger.critical("Start requesting")
if len(duet.store) > 0:
    t = duet.store["size_1"].get(
        request_block=True,
        timeout_secs=15,
        name="age_data",
        reason="I want to see the age data",
        delete_obj=False
    )
    sy.logger.critical("Finished requesting")

In [ ]:
print(t)

In [ ]:
# lets get some references to our data owners Duet torch and torchvision
torch = duet.torch
torchvision = duet.torchvision

# these are the same as the original mnist example
transforms = torchvision.transforms
datasets = torchvision.datasets
nn = torch.nn
F = torch.nn.functional
optim = torch.optim
StepLR = torch.optim.lr_scheduler.StepLR

In [ ]:
import torch as th
import torchvision as tv

In [ ]:
# we need some transforms for our MNIST data set
local_transform_1 = tv.transforms.ToTensor()  # this converts PIL images to Tensors
local_transform_2 = tv.transforms.Normalize(0.1307, 0.3081)  # this normalizes the dataset

# compose our transforms
local_transforms = tv.transforms.Compose([local_transform_1, local_transform_2])

In [ ]:
# Training settings from original MNIST example command line args
args = {
    "batch_size": 64,
    "test_batch_size": 1000,
    "epochs": 14,
    "lr": 1.0,
    "gamma": 0.7,
    "no_cuda": False,
    "dry_run": False,
    "seed": 42, # the meaning of life
    "log_interval": 10,
    "save_model": False,
}

In [ ]:
test_kwargs = {
    "batch_size": args["test_batch_size"],
}

# this is our carefully curated test data which represents the goal of our problem domain
test_data = tv.datasets.MNIST('../data', train=False, download=True, transform=local_transforms)
test_loader = th.utils.data.DataLoader(test_data,**test_kwargs)

In [ ]:
test_data_length = len(test_loader.dataset)
print(test_data_length)

# test_data_length_ptr = duet.syft.lib.python.Int(test_data_length)
# print(test_data_length_ptr)

In [ ]:
print(test_data, type(test_data))

The "duet" variable is now your reference to a whole world of remote operations including supported libraries like torch.

In [ ]:
# # TODO replace with local inference so this doesn't need to be on the DO side
# test_data_ptr = torchvision.datasets.MNIST('../data', train=False, download=True, transform=transforms)
# print(test_data_ptr)

# test_loader_ptr = torch.utils.data.DataLoader(test_data_ptr,**test_kwargs)
# print(test_loader_ptr)
# # TODO

In [ ]:
class SyNet(sy.Module):
    def __init__(self, torch_ref):
        super(SyNet, self).__init__(torch_ref=torch_ref)
#         print(f"modal is local: {self.is_local} running with torch_ref", type(torch_ref))
        self.conv1 = self.torch_ref.nn.Conv2d(1, 32, 3, 1)
        self.conv2 = self.torch_ref.nn.Conv2d(32, 64, 3, 1) 
        self.dropout1 = self.torch_ref.nn.Dropout2d(0.25)
        self.dropout2 = self.torch_ref.nn.Dropout2d(0.5)
        self.fc1 = self.torch_ref.nn.Linear(9216, 128)
        self.fc2 = self.torch_ref.nn.Linear(128, 10)

    def forward(self, x):
#         print(f"modal is local: {self.is_local} forward called with {self}, {x}")
#         print(f"modal is local: {self.is_local} getting data", type(x))
#         print(f"modal is local: {self.is_local} using layer conv1", type(self.conv1))
        x = self.conv1(x)
        x = self.torch_ref.nn.functional.relu(x)
        x = self.conv2(x)
        x = self.torch_ref.nn.functional.relu(x)
        x = self.torch_ref.nn.functional.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = self.torch_ref.flatten(x, 1)
        x = self.fc1(x)
        x = self.torch_ref.nn.functional.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = self.torch_ref.nn.functional.log_softmax(x, dim=1)
#         print(f"modal is local: {self.is_local} producing output of type", type(output))
        return output

In [ ]:
local_model = SyNet(th)
local_model.debug_sum_layers()

In [ ]:
# local_model.conv2.state_dict()["weight"]

In [ ]:
model = local_model.send(duet)

In [ ]:
# model.conv2.state_dict().get()["weight"]

In [ ]:
# print(remote.is_local)
# print(remote.duet)
# print(type(remote.torch_ref))

In [ ]:
# # remote weights will be 0
# remote.conv1.state_dict().get()

In [ ]:
# print(model.modules)
# print(remote.modules)

In [ ]:
# lets see if our Data Owner has CUDA
has_cuda = False
has_cuda_ptr = torch.cuda.is_available()
has_cuda = bool(has_cuda_ptr.get(
    request_block=True,
    name="cuda_is_available",
    reason="To run test and inference locally",
    timeout_secs=30,  # change to something slower
))
print(has_cuda)

In [ ]:
use_cuda = not args["no_cuda"] and has_cuda
torch.manual_seed(args["seed"])

device = torch.device("cuda" if use_cuda else "cpu")
print(f"DO device is {device.type.get()}")

In [ ]:
# if we have CUDA lets send our model to the GPU
if has_cuda:
    model.cuda(device)
else:
    model.cpu()

In [ ]:
# lets get our parameters for optimization
# params_list required for remote list concatenation
params = model.parameters()

In [ ]:
optimizer = optim.Adadelta(params, lr=args["lr"])
print(optimizer, type(optimizer))

In [ ]:
scheduler = StepLR(optimizer, step_size=1, gamma=args["gamma"])
print(scheduler, type(scheduler))

In [ ]:
# model.conv2.state_dict().get()["weight"]

In [ ]:
# now can define a simple training loop very similar to the original PyTorch MNIST example
# @sy.logger.catch
def train(args, model, device, train_loader, optimizer, epoch, train_data_length):
    # + 0.5 lets us math.ceil without the import
    train_batches = round((train_data_length / args["batch_size"]) + 0.5)
    train_batches = 100
    print(f"> Running train in {train_batches} batches")
    if model.is_local:
        print("Training requires remote model")
        return
    model.train()

    for batch_idx, data in enumerate(train_loader):
        data_ptr, target_ptr = data[0], data[1]
        optimizer.zero_grad()
        output = model(data_ptr)
        loss = F.nll_loss(output, target_ptr)
        loss.backward()
        optimizer.step()
        loss_item = loss.item()
        train_loss = duet.syft.lib.python.Float(0)
        train_loss += loss_item
        if batch_idx % args["log_interval"] == 0:
            local_loss = None
#             local_loss = loss_item.get(
#                 name="loss",
#                 reason="To evaluate training progress",
#                 request_block=True,
#                 timeout_secs=30
#             )
            if local_loss is not None:
                print("Train Epoch: {} {} {:.4}".format(epoch, batch_idx, local_loss))
            else:
                print("Train Epoch: {} {} ?".format(epoch, batch_idx))
            if args["dry_run"]:
                break
        if batch_idx >= train_batches - 1:
            print("batch_idx >= train_batches, breaking")
            break

In [ ]:
# TODO replace with local inference and local test set
# the same for our test training loop except we will need to send our data over for inference
# @sy.logger.catch
def test_local(remote_model, test_loader, test_data_length):
    # download remote model
    if not remote_model.is_local:
        local_model = remote_model.get()

#         local_model = remote_model.get(
#             request_block=False,
#             name="model_download",
#             reason="test evaluation",
#             timeout_secs=30
#         )
    else:
        local_model = remote_model
        
    # visually check the weights have changed
    local_model.debug_sum_layers()
    # + 0.5 lets us math.ceil without the import
    test_batches = round((test_data_length / args["test_batch_size"]) + 0.5)
    print(f"> Running test_local in {test_batches} batches")
    local_model.eval()
    test_loss = 0.0
    correct = 0.0

    with th.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            output = local_model(data)
            iter_loss = th.nn.functional.nll_loss(output, target, reduction='sum').item()
            test_loss = test_loss + iter_loss
            pred = output.argmax(dim=1)
            total = pred.eq(target).sum().item()
            correct += total
            if args["dry_run"]:
                break
                
            if batch_idx >= test_batches - 1:
                print("batch_idx >= test_batches, breaking")
                break

    accuracy = correct / test_data_length
    print(f"Test Set Accuracy: {100 * accuracy}%")

In [ ]:
# we need some transforms for our MNIST data set
transform_1 = torchvision.transforms.ToTensor()  # this converts PIL images to Tensors
transform_2 = torchvision.transforms.Normalize(0.1307, 0.3081)  # this normalizes the dataset
print(type(transform_1), type(transform_2))

remote_list = duet.syft.lib.python.List()
remote_list.append(transform_1)
remote_list.append(transform_2)

# compose our transforms
transforms = torchvision.transforms.Compose(remote_list)

# The DO has kindly let us initialise a DataLoader for their training set
train_kwargs = {
    "batch_size": args["batch_size"],
}
train_data_ptr = torchvision.datasets.MNIST('../data', train=True, download=True, transform=transforms)
print(train_data_ptr)
train_loader_ptr = torch.utils.data.DataLoader(train_data_ptr,**train_kwargs)
print(train_loader_ptr)

In [ ]:
def get_train_length(train_data_ptr):
    train_length_ptr = train_data_ptr.__len__()
    train_data_length = train_length_ptr.get(
        request_block=True,
        name="train_size",
        reason="To write the training loop",
        timeout_secs=30,
    )
    return train_data_length

try:
    if train_data_length is None:
        train_data_length = get_train_length(train_data_ptr)
except NameError:
        train_data_length = get_train_length(train_data_ptr)

print(f"Training Dataset size is: {train_data_length}")

In [ ]:
%%time
import time
# sequential CPU times: user 39.2 s, sys: 390 ms, total: 39.6s per epoch
# sy.module CPU times: user 1min 28s, sys: 645 ms, total: 1min 28s per epoch
# vanilla class CPU times: user 3min 5s, sys: 3.01 s, total: 3min 8s per epoch

args["dry_run"] = False
sy.logger.trace("Start Training")
for epoch in range(1, args["epochs"] + 1):
    epoch_start = time.time()
    print(f"Epoch: {epoch}")
    train(args, model, device, train_loader_ptr, optimizer, epoch, train_data_length)
    test_local(model, test_loader, test_data_length)  # real local data and model
    scheduler.step()
    epoch_end = time.time()
    print(f"Epoch time: {int(epoch_end - epoch_start)} seconds")
    break
sy.logger.trace("Finish Training")

In [ ]:
print("Done")

In [ ]:
print(model.is_local)
# model.debug_sum_layers()

In [ ]:
# l = model.get()

In [ ]:
# l.debug_sum_layers()

# PART 3: Inference

In [ ]:
import matplotlib.pyplot as plt
def draw_image_and_label(image, label):
    fig = plt.figure()
    plt.tight_layout()
    plt.imshow(image, cmap="gray", interpolation="none")
    plt.title("Ground Truth: {}".format(label))
    
def prep_for_inference(image):
    image_batch = image.unsqueeze(0).unsqueeze(0)
    image_batch = image_batch * 1.0
    return image_batch

In [ ]:
def classify_local(image, model):
    if not model.is_local:
        print("model is remote try .get()")
        return -1, th.Tensor([-1])
    image_tensor = th.Tensor(prep_for_inference(image))
    output = model(image_tensor)
    preds = th.exp(output)
    local_y = preds
    local_y = local_y.squeeze()
    pos = local_y == max(local_y)
    index = th.nonzero(pos, as_tuple=False)
    class_num = index.squeeze()
    return class_num, local_y

In [ ]:
def classify_remote(image, model):
    if model.is_local:
        print("model is local try .send()")
        return -1, th.Tensor([-1])
    image_tensor_ptr = torch.Tensor(prep_for_inference(image))
    output = model(image_tensor_ptr)
    preds = torch.exp(output)
    preds_result = preds.get(
        request_block=True,
        name="inference",
        reason="To see a real world example of inference",
        timeout_secs=10
    )
    if preds_result is None:
        print("No permission to do inference, request again")
        return -1, th.Tensor([-1])
    else:
        local_y = th.Tensor(preds_result)
        local_y = local_y.squeeze()
        pos = local_y == max(local_y)
        index = th.nonzero(pos, as_tuple=False)
        class_num = index.squeeze()
        return class_num, local_y

In [ ]:
# lets grab something from the test set
import random
total_images = test_data_length # 10000
index = random.randint(0, total_images)
print("Random Test Image:", index)
count = 0
batch = index // test_kwargs["batch_size"]
batch_index = index % int(total_images / len(test_loader))
for tensor_ptr in test_loader:
    data, target = tensor_ptr[0], tensor_ptr[1]
    if batch == count:
        break
    count += 1

print(f"Displaying {index} == {batch_index} in Batch: {batch}/{len(test_loader)}")
image_1 = data[batch_index].reshape((28, 28))
label_1 = target[batch_index]
draw_image_and_label(image_1, label_1)

In [ ]:
# classify remote
sy.logger.trace("Before running classify")
class_num, preds = classify_remote(image_1, model)
print(f"Prediction: {class_num} Ground Truth: {label_1}")
print(preds)

In [ ]:
local_model = model.get()

In [ ]:
# classify local
sy.logger.trace("Before running classify")
class_num, preds = classify_local(image_1, local_model)
print(f"Prediction: {class_num} Ground Truth: {label_1}")
print(preds)

In [ ]:
from PIL import Image, ImageEnhance
import PIL.ImageOps    

import os
def classify_url_image(image_url):
    filename = os.path.basename(image_url)
    !curl -O $image_url
    im = Image.open(filename)
    im = PIL.ImageOps.invert(im)
#     im = im.resize((28,28), Image.ANTIALIAS)
    im = im.convert('LA')
    enhancer = ImageEnhance.Brightness(im)
    im = enhancer.enhance(3)


    print(im.size)
    fig = plt.figure()
    plt.tight_layout()
    plt.imshow(im, cmap="gray", interpolation="none")
    
    # classify local
    class_num, preds = classify_local(image_1, local_model)
    print(f"Prediction: {class_num}")
    print(preds)

In [ ]:
image_url = "https://raw.githubusercontent.com/kensanata/numbers/master/0018_CHXX/0/number-100.png"
classify_url_image(image_url)